<html><font size=6><B>3η εργαστηριακή άσκηση</B><BR>
    <BR><B>Link prediction - Πρόβλεψη συνδέσμων</B></BR>
    </font></html>

<html><font size=4><B>1. Κατασκευή γράφου και προεργασία για την πρόβλεψη συνδέσμων</B></font>
    <li><B>1(i).</B> Χρησιμοποιήστε το dataset <a href="http://konect.cc/networks/dbpedia-similar/"> Similarities (DBpedia)</a>, για να κατασκευάσετε τον μη κατευθυνόμενο (undirected) γράφο $G$ <a href="https://dl.acm.org/doi/10.1145/2487788.2488173">[1]</a>,<a href="https://link.springer.com/chapter/10.1007/978-3-540-76298-0_52">[2]</a>. Οι κόμβοι του γράφου αναπαριστούν σελίδες της Wikipedia και οι ακμές σχέση ομοιότητας μεταξύ τους. Στην σελίδα <a href="http://konect.cc/networks/dbpedia-similar/"> KONECT (DBpedia)</a> μπορείτε να βρείτε πληροφορίες για τις ιδιότητες του γράφου.</a><BR>
        <B><U>Υπόδειξη:</U></B> Μπορείτε να ελέγξετε προγραμματιστικά αν το dataset περιέχει διπλότυπα των ακμών με την ίδια ή αντίστροφη σειρά (π.χ. ακμές (a,b) και (b,a)), και να λάβετε υπ' όψιν σας αυτήν σας την παρατήρηση, όπου τυχόν χρειαστεί, στην ερώτηση 2.<BR></html>

 <li><B>1(ii).</B> Εμφανίστε τον αριθμό των κόμβων, τον αριθμό των ακμών, τον μέσο βαθμό των κόμβων και τον αριθμό των συνεκτικών συνιστωσών (connected components) για τον γράφο $G$.</li><BR>

In [4]:
import networkx as nx
import pandas as pd
import numpy as np

In [5]:
G: nx.Graph = nx.read_edgelist('dbpedia-similar/out.dbpedia-similar', comments='%')
G.graph['name'] = 'DBpedia Similarities'

In [6]:
# remove self loops
self_loops = nx.selfloop_edges(G)
self_loops_set = set(self_loops)
print(f"Number of self loops removed: {len(self_loops_set)}. Edges removed: {self_loops_set}")
G.remove_edges_from(self_loops_set)


Number of self loops removed: 1. Edges removed: {('420', '420')}


In [7]:
def average_degree(G: nx.Graph):
    sum_of_node_degrees = sum([v for k,v in G.degree()])
    node_count = G.number_of_nodes()
    avg_degree = sum_of_node_degrees/node_count
    return avg_degree

initial_node_count = G.number_of_nodes()
initial_edge_count = nx.number_of_edges(G)
initial_avg_node_degree = average_degree(G)
initial_connected_components = nx.number_connected_components(G)

print(f"Graph: {G.graph['name']}")
print(f"Initial node count: {initial_node_count}")
print(f"Initial edge count: {initial_edge_count}")
print(f"Initial average node degree: {initial_avg_node_degree:.3f}")
print(f"Initial number of connected components: {initial_connected_components}")

Graph: DBpedia Similarities
Initial node count: 430
Initial edge count: 564
Initial average node degree: 2.623
Initial number of connected components: 95


<li><B>1(iii). </B>Υπολογίστε το σύνολο των ασύνδετων ζευγών κόμβων (unconnected pairs) στον γράφο χρησιμοποιώντας τον πίνακα γειτνίασης (adjacency matrix). Δώστε επίσης τον αριθμό τους. Λάβετε υπ' όψιν σας ότι ο γράφος είναι μη κατευθυνόμενος και δεν χρειάζεται να διατρέξετε ολόκληρο τον πίνακα γειτνίασης.</li><BR>

In [8]:
# since graph is undirected, it is symmetrical therefore we just need to check half of the matrix.
# also the diagonal has 0s, since there are no self-loops (one existed initially and was removed)

adj_matrix: np.matrix = nx.to_numpy_matrix(G)

lower_matrix: np.ndarray = np.tril(adj_matrix)

In [9]:
xs_array, ys_array = np.where(lower_matrix == 0)
unconnected_pairs_idxs = list(zip(xs_array, ys_array))
unconnected_pairs_idxs = [(x,y) for x,y in unconnected_pairs_idxs if (y<x)] # filter out the diagonal elements and everything above the diagonal
nodes = np.array(list(G.nodes))
unconnected_pairs_u = nodes[[x for x, _ in unconnected_pairs_idxs]]
unconnected_pairs_v = nodes[[y for _, y in unconnected_pairs_idxs]]
unconnected_pairs = set(zip(unconnected_pairs_u, unconnected_pairs_v))
num_of_unconnected_pairs = len(unconnected_pairs)
print(f"Number of unconnected pairs: {num_of_unconnected_pairs}")


Number of unconnected pairs: 91671


<li><B>1(iv).</B> Υπολογίστε το σύνολο των ακμών (removable edges) που μπορούν να αφαιρεθούν από τον γράφο χωρίς να υπάρχει κατάτμηση (splitting) του γράφου (δηλ. ο αριθμός των συνεκτικών συνιστωσών θα πρέπει να παραμένει αμετάβλητος). Υπόδειξη: Ελέγχετε επίσης ότι με αφαίρεση ακμής δεν μειώνεται ο αριθμός των κόμβων (εφόσον δεν υπάρχει ξεχωριστό αρχείο μόνο με κόμβους στο dataset). Δώστε επίσης τον αριθμό των ακμών (removable edges) που μπορούν να αφαιρεθούν υπό αυτές τις προϋποθέσεις.</li><BR>

In [10]:
def graph_edge_remove(G: nx.Graph, edge_to_remove: tuple) -> nx.Graph:
    G.remove_edge(*edge_to_remove)
    return G

def graph_edge_add(G: nx.Graph, edge_to_add: tuple) -> nx.Graph:
    G.add_edge(*edge_to_add)
    return G

In [11]:
removable_edges = set()

G_removed_edges: nx.Graph = G.copy()

for edge in G.edges:
    graph_edge_remove(G_removed_edges, edge)
    new_conn_components = nx.number_connected_components(G_removed_edges)
    new_G_nodes = G_removed_edges.number_of_nodes()
    if (new_G_nodes == initial_node_count) and (new_conn_components == initial_connected_components):
        removable_edges.add(edge)

    else:
        graph_edge_add(G_removed_edges, edge)

print(f"The number of removable edges of the graph are: {len(removable_edges)}")    

The number of removable edges of the graph are: 229


<li><B>1(vα).</B> Κατασκευάστε dataframe που συνενώνει τις απαντήσεις των ερωτημάτων 1(iii) και 1(iv) (unconnected+removables). Χρησιμοποιήστε την μεταβλητή $link$ που θα παίρνει τιμή 1 ή 0 ανάλογα με την ύπαρξη συνδέσμου ή μη. To dataframe αυτό συγκεντρώνει τα θετικά και αρνητικά δείγματα για την πρόβλεψη των μελλοντικών συνδέσεων (link: target variable). <BR>
    <li><B>1(vβ).</B> Κατασκευάστε τον γράφο $G'$ που προκύπτει από την αφαίρεση από τον γράφο $G$ των ακμών του ερωτήματος 1(iv). <BR>
    <li><B>1(vγ).</B> Εμφανίστε τον αριθμό των κόμβων, τον αριθμό των ακμών, τον μέσο βαθμό των κόμβων και τον αριθμό των συνεκτικών συνιστωσών (connected components) για τον γράφο $G'$.</li><BR>

In [12]:
df = pd.DataFrame({
    'Edge': list(removable_edges) + list(unconnected_pairs),
    'Link': [1] * len(removable_edges) + [0] * len(unconnected_pairs)
    }
)

In [13]:
df.head()

,Edge,Link
0,"(343, 344)",1
1,"(32, 21)",1
2,"(338, 341)",1
3,"(19, 20)",1
4,"(68, 93)",1


In [14]:
G_reduced = G.copy()
G_reduced.graph['name'] = f"{G.graph['name']} reduced"
G_reduced.remove_edges_from(removable_edges)

In [15]:
G_reduced_node_count = G_reduced.number_of_nodes()
G_reduced_edge_count = nx.number_of_edges(G_reduced)
G_reduced_avg_node_degree = average_degree(G_reduced)
G_reduced_connected_components = nx.number_connected_components(G_reduced)

print(f"Graph: {G_reduced.graph['name']}")
print(f"Node count: {G_reduced_node_count}")
print(f"Edge count: {G_reduced_edge_count}")
print(f"Average node degree: {G_reduced_avg_node_degree:.3f}")
print(f"Number of connected components: {G_reduced_connected_components}")

Graph: DBpedia Similarities reduced
Node count: 430
Edge count: 335
Average node degree: 1.558
Number of connected components: 95


   # Εισαγωγή σε similarity-based μετρικές για Link prediction
 - Η μετρική Jaccard Coefficient (JC) των κόμβων $u$ και $v$ ορίζεται ως $\frac{|\Gamma(u) \cap \Gamma(v)|}{|\Gamma(u) \cup \Gamma(v)|}$, όπου ${\Gamma(u)}$ είναι το σύνολο των γειτονικών κόμβων του κόμβου ${u}$.  
- Η μετρική Preferential Αttachment (PA) των κόμβων $u$ και $v$ ορίζεται ως $|\Gamma(u)| |\Gamma(v)|$, όπου $\Gamma(u)$ είναι το σύνολο των γειτονικών κόμβων του $u$.
- Η μετρική Resource Allocation (RA) των κόμβων $u$ και $v$ ορίζεται ως $\sum_{w \in \Gamma(u) \cap \Gamma(v)} \frac{1}{|\Gamma(w)|}$, όπου $\Gamma(u)$ είναι το σύνολο των γειτονικών κόμβων του $u$.

2(i). Για τον γράφο $G'$, που κατασκευάσατε στο ερώτημα 1(vβ), υπολογίστε την μετρική <B>Jaccard Coefficient (JC)</B>.  <a href="https://networkx.org/documentation/stable/reference/algorithms/link_prediction.html?highlight=link%20prediction">Χρησιμοποιήστε την υλοποιημένη συνάρτηση του λογισμικού NetworkX.</a>

2(ii). Δώστε τις $k$ ακμές με την υψηλότερη τιμή μετρικής $JC$, όπου $k$ ο $Α.Μ.$ σας $mod$ $10$. Για $Α.Μ.$ λήγοντα σε $0$ δώστε τις $10$ ακμές με την υψηλότερη τιμή μετρικής.

(Για την σύγκριση μεταξύ διαφορετικών μετρικών προτιμάται η εύρεση των $k$ επικρατέστερων συνδέσμων, αφού οι μετρικές επιστρέφουν διαφορετικό εύρος τιμών.)

2(iii). Προσδιορίστε για την μέτρηση της απόδοσης της μετρικής $JC$ τα μεγέθη $Precision$, $Recall$ και $Accuracy$ και αναπαραστήστε τα γραφικά για τιμές κατωφλίου: $0.1$, $0.15$, $0.25$, $0.5$, $1.0$.  <BR><BR>
    $Precision=\frac{TP}{TP+FP}$, όπου $TP$: αληθώς θετικά αποτελέσματα, $FP$: ψευδώς θετικά αποτελέσματα <BR>
    $Recall=\frac{TP}{TP+FN}$, όπου $TP$: αληθώς θετικά αποτελέσματα, $FN$: ψευδώς αρνητικά αποτελέσματα</li> 
    $Accuracy=\frac{TP+TN}{TP+FN+TN+FP}$, όπου $TP$: αληθώς θετικά αποτελέσματα, $TN$: αληθώς αρνητικά αποτελέσματα, $FN$: ψευδώς αρνητικά αποτελέσματα, $FP$: ψευδώς θετικά αποτελέσματα</li><BR><BR>
        <B><U>Υπόδειξη:</U></B>
        Θετικό θεωρήστε ένα αποτέλεσμα, αν η μετρική $JC$ υπερβαίνει μια τιμή κατωφλίου (>=). Αρνητικό θεωρήστε το στην αντίθετη περίπτωση.
        <li>Αληθώς θετικά είναι τα αποτελέσματα που βγήκαν θετικά (η τιμή της $JC$ υπερβαίνει την τιμή κατωφλίου) και πράγματι θα υπάρξουν οι συνδέσεις που περιγράφουν (ερ. 1(iv)).</li>
        <li>Αληθώς αρνητικά είναι τα αποτελέσματα που βγήκαν αρνητικά και πράγματι δεν θα υπάρξουν οι συνδέσεις που περιγράφουν (ερ. 1(iii)).</li>
        <li>Ψευδώς θετικά είναι τα αποτελέσματα που βγήκαν θετικά, ενώ δεν θα υπάρξουν οι συνδέσεις που περιγράφουν.</li>
        <li>Ψευδώς αρνητικά είναι τα αποτελέσματα που βγήκαν αρνητικά, ενώ θα υπάρξουν οι συνδέσεις που περιγράφουν.</li>
        <li>Το άθροισμα $TN+FP$ πρέπει κάθε φορά να σας δίνει όλες τις ασύνδετες ακμές του γράφου $G$.</li>   
<li>Από το ερώτημα 1(iii) οι ακμές που υπολογίσατε είναι αυτές για τις οποίες δεν θα υπάρξουν συνδέσεις.</li> <li>Από το ερώτημα 1(iv) οι ακμές που υπολογίσατε είναι αυτές για τις οποίες θα υπάρξουν συνδέσεις.</li><BR></html>

In [16]:
k = 3400131 % 10  # k=1 for the selected student number
jc = nx.jaccard_coefficient(G_reduced)
jc_sorted = sorted(jc, key=lambda x: -x[2])
jc_for_k = jc_sorted[:k]
print(f"The {k} edge(s) with the largest jaccard coefficient is/are:")
for item in jc_for_k:
    print(f"Edge: ({item[0]},{item[1]}), JC: {item[-1]}")

The 1 edge(s) with the largest jaccard coefficient is/are:
Edge: (385,386), JC: 1.0


In [17]:
df[df['Edge'] == ('105', '110')]['Link'][0] == 1

KeyError: 0

In [ ]:

def JCfunction(G, threshold_jc):
    ebunch_edges = list(map(lambda x: (x[0], x[1]), df['Edge']))
    jc = nx.jaccard_coefficient(G, ebunch_edges)

    true_links, pred_links = [], []
    for u, v, jcoef in jc:
        pred_link = jcoef >= threshold_jc
        pred_links.append(pred_link)

    true_links = list(df['Link'])

    TP, TN, FP, FN = 0, 0, 0, 0
    
    for i, truth in enumerate(true_links):
        if truth == 1 and truth == pred_links[i]:
            TP += 1
        elif truth == 0 and truth == pred_links[i]:
            TN += 1
        elif truth == 1 and truth != pred_links[i]:
            FN += 1
        else:
            FP += 1

    precision_jc = TP / (TP + FP)
    recall_jc = TP / (TP + FN)
    accuracy_jc = (TP + TN) / (TP + FP + TN + FN)

    return recall_jc, precision_jc, accuracy_jc

In [ ]:
thr_range = (0.1, 0.15, 0.25, 0.5, 1.0)
pr, rc, acc = [], [], []
print('Jaccard coefficients - Link prediction')
for jc_threshold in thr_range:
    print(f'Threshold: {jc_threshold}')
    recall, precision, accuracy = JCfunction(G_reduced, jc_threshold)
    pr.append(precision)
    rc.append(recall)
    acc.append(accuracy)
    print(f'\tPrecision: {round(precision, 3)}\n\tRecall: {round(recall, 3)}\n\tAccuracy: {round(accuracy, 3)}')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(thr_range, pr, label='Precision')
plt.plot(thr_range, rc, label='Recall')
plt.plot(thr_range, acc, label='Accuracy')
plt.legend()
plt.show()

From the printed results and the plot, we can get to the conclusion that the accuracy metric is close to the value of 1 regardless of the selection of jaccard coefficient threshold. This is largely due to the imbalanced nature of the two classes (linked and no linked), where the majority of the nodes are not linked together (no edge exists). Therefore, precision and recall seem to be a better fit as metrics for the given task. 

From the plot, the tradeoff between precision and recall can be clearly observed. For low thresholds, precision is low, which means that there are a lot of FP (false positives), i.e. given that the threshold is low, several edges are classified as linked, where in reality they aren't linked. At the same time, recall is larger compared to other threshold values, which means that FN (false negatives) are low, i.e. since most of the edges are classified/predicted as Positive (linked), then the number of wrong predictions of the Negative (no link) class, is lower.

The conclusions for higher values of the threshold are similar. 

<html><font size=4><B>3. Πρόβλεψη συνδέσμων βάσει similarity-based μετρικών</B></font>    
        <li><B>3(i).</B> Προτείνετε μια τουλάχιστον μέθοδο, για να εξερευνήσουμε και να εκφράσουμε μέσα από τα πακέτα NumPy ή SciPy ή Pandas την συσχέτιση (correlation) μεταξύ των τιμών των μετρικών <B>Jaccard Coefficient</B>, <B>Preferential Attachment</B>, <B>Resource Allocation</B> για τον μειωμένο γράφο $G'$. Σημ.: Δεν χρειάζεται κάποια υλοποίηση, μόνο αναφορά της μεθόδου συσχέτισης και επεξήγηση της επιλογής σας.
     <li><B>3(ii).</B> Εργαστείτε πάνω σε <B><U>αντίγραφο</U></B> του dataframe που κατασκευάσατε στο ερώτημα 1(vα) (unconnected+removables). Δημιουργήστε στήλη (column) με τις ακμές του γράφου και θέστε την ως index του dataframe. Στο dataframe θα προσθέσετε ως στήλες τις τιμές των μετρικών (<B>Jaccard Coefficient</B>, <B>Preferential Attachment</B>, <B>Resource Allocation</B>) για τον μειωμένο γράφο $G'$. Οι τιμές των μετρικών μπορούν να υπολογιστούν με τις συναρτήσεις του πακέτου NetworkX.</li>   </font></html>
    
    |       | link  |Jacc.Coeff.|Pr.Attachment| Resource Allocation|
    |-------|-------|-----------|-------------|--------------------|
    |(1, 6) |  0    |  0.333333 |     32      |    0.700000        |
    |(1, 46)|  0    |  0.200000 |      8      |    0.166667        |

For the correlation calculation between the metrics of jaccard coefficient, preferential attachment and resource allocation, the numpy method `np.corrcoef()` can be used. The parameters passed to the method are rowvar=False, for taking rows as records and columns as features as well as a transpose of a multidimensional matrix.

Also, in the pandas library, the `DataFrame.corr()` method computes pairwise Pearson correlations between rows or columns of a DataFrame. By default, the function calculates the Pearson correlation coefficient between the numeric columns of the DataFrame. We can also specify which columns to include in the calculation by passing a list of column names to the `DataFrame.corr()` method, eg. `correlation_matrix = df[["col1", "col2"]].corr()`.

In [18]:
df.head()

,Edge,Link
0,"(343, 344)",1
1,"(32, 21)",1
2,"(338, 341)",1
3,"(19, 20)",1
4,"(68, 93)",1


In [19]:
#replicate of the dataframe "unconnected+removables"---dataframe will be used again in (4)
import copy

df2 = copy.deepcopy(df)
df2.set_index("Edge", inplace=True)
df2.rename_axis(None, inplace=True)
df2.head()

,Link
"(343, 344)",1
"(32, 21)",1
"(338, 341)",1
"(19, 20)",1
"(68, 93)",1


In [20]:
# df.sort_values(by='Edge', key=lambda col: col.map(lambda x: (int(x[0]), int(x[1]))))

In [21]:
ebunch_edges = list(map(lambda x: (x[0], x[1]), df['Edge']))
#dataframe for similarity-based measurement values: df2

#Jaccard Coefficient
jc = nx.jaccard_coefficient(G_reduced, ebunch_edges)
df2['Jacc.Coeff.'] = [x[-1] for x in jc]

#Preferential Attachment
pa = nx.preferential_attachment(G_reduced, ebunch_edges)
df2['Pr.Attachment'] = [x[-1] for x in pa]

#Resource Allocation
ra = nx.resource_allocation_index(G_reduced, ebunch_edges)
df2['Resource Allocation'] = [x[-1] for x in ra]


df2.head()

,Link,Jacc.Coeff.,Pr.Attachment,Resource Allocation
"(343, 344)",1,1.0,1,0.066667
"(32, 21)",1,0.0,18,0.000000
"(338, 341)",1,1.0,1,0.066667
"(19, 20)",1,0.0,12,0.000000
"(68, 93)",1,0.0,2,0.000000


In [22]:
df2.corr()

,Link,Jacc.Coeff.,Pr.Attachment,Resource Allocation
Link,1.000000,0.543554,0.011689,0.254448
Jacc.Coeff.,0.543554,1.000000,-0.015560,0.630806
Pr.Attachment,0.011689,-0.015560,1.000000,0.030255
Resource Allocation,0.254448,0.630806,0.030255,1.000000


It's interesting to notice here that Preferential Attachement has very low correlation with Link, Jaccard Coefficient and Resource Allocation. 
We might expect, based on this observation, that the contribution of the Preferential Attachement feature to the link prediction task will be limited.

In [23]:
# df2.sort_index(key=lambda col: col.map(lambda x: (int(x[0]), int(x[1]))))

<li><B>3(iiiα).</B> Χρησιμοποιήστε τον Random Forest Classifier και το αντίγραφο dataframe που κατασκευάσατε στο ερώτημα 3(ii), για να προβλέψετε συνδέσεις. Συμπληρώστε στον κώδικα: τα features ($X$), την target variable ($y$) και τα ορίσματα της συνάρτησης $fit$ για το training του μοντέλου.</li>
<li><B>3(iiiβ).</B> Δώστε τις ($A.M$ $mod$ $10$) πρώτες προβλέψεις και την ακρίβεια ($accuracy$) του μοντέλου. </li>
<li><B>3(iiiγ).</B>Υπολογίστε την πιθανότητα να συνδεθούν οι κόμβοι ($1$,$47$) βάσει του μοντέλου σας. </li>  

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
 
X = df2[['Jacc.Coeff.', 'Pr.Attachment', 'Resource Allocation']].to_numpy() # features
y = df2['Link'].to_numpy() # target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 0)

print(f'X shape: {X.shape}, type: {X.dtype}')
print(f'\ttrain: {X_train.shape}')
print(f'\ttest : {X_test.shape}')

print(f'y shape: {y.shape}, type: {y.dtype}')
print(f'\ttrain: {y_train.shape}')
print(f'\ttest : {y_test.shape}')


X shape: (91900, 3), type: float64
	train: (61573, 3)
	test : (30327, 3)
y shape: (91900,), type: int64
	train: (61573,)
	test : (30327,)


In [25]:
classifier = RandomForestClassifier()   

from sklearn.preprocessing import MinMaxScaler 
mm_scaler = MinMaxScaler() 
X_train_scaled = mm_scaler.fit_transform(X_train)
X_test_scaled = mm_scaler.transform(X_test)

parameters = {'n_estimators' : [10,50,100], 'max_depth' : [5,10,15]} 
model = GridSearchCV(classifier, param_grid = parameters, scoring='f1') 

In [26]:
model.fit(X_train_scaled, y_train) # train the model

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15],
                         'n_estimators': [10, 50, 100]},
             scoring='f1')

Given that the dataset is imbalanced, we selected the F1-score as an appropriate scoring metric for the classification task instead of accuracy. The accuracy is expected to be close to 1 without being indicative of the predictive power of our model.

In [27]:
print(f'The best estimator found from Grid Search is {model.best_estimator_}')
print(f'Parameters: {model.best_params_}')
print(f'F1-score = {model.best_score_}')

The best estimator found from Grid Search is RandomForestClassifier(max_depth=5, n_estimators=50)
Parameters: {'max_depth': 5, 'n_estimators': 50}
F1-score = 0.7087168741573409


In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

AM = 3400131
k = AM % 10 # AM mod 10

predictions = model.predict(X_test_scaled)       
k_first_preds = predictions[:k] #get (A.M. last digit) first predictions
k_first_labels = y_test[:k]

print(f'The first {k} prediction(s) is/are: {list(k_first_preds)}\nThe first {k} true label(s) is/are: {list(k_first_labels)}\n')

acc = accuracy_score(y_test, predictions)
pr = precision_score(y_test, predictions)
rc = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print('Prediction metrics:')
print(f'Accuracy : {round(acc, 4)}\nPrecision: {round(pr, 4)}\nRecall   : {round(rc, 4)}\nF1-score : {round(f1, 4)}')

The first 1 prediction(s) is/are: [0]
The first 1 true label(s) is/are: [0]

Prediction metrics:
Accuracy : 0.9986
Precision: 0.9667
Recall   : 0.4203
F1-score : 0.5859


In [30]:
u, v = '1', '47' # nodes for which to make an link prediction (whether an edge exists in graph or no)

train_sample = df2[df2.index == (v, u)].iloc[:, 1:].to_numpy()
train_sample = np.reshape(train_sample, (1, -1)) # reshape to (3,1) numpy array

predict_proba = model.predict_proba(train_sample)

print(f'The probability that nodes {u} and {v} are connected/linked with an edge is predicted by the classifier to be: {predict_proba[0, 1]}')

The probability that nodes 1 and 47 are connected/linked with an edge is predicted by the classifier to be: 0.005522923746997231


<html><font size=4><B>4. Πρόβλεψη συνδέσμων με embedding βασισμένο σε τυχαίους περιπάτους (Random Walks) </B></font><BR><BR>
    <li><B>4(i).</B> Εξάγετε τα χαρακτηριστικά (features) του γράφου $G'$, που προέκυψε από την αφαίρεση των ακμών. Για τον σκοπό αυτό χρησιμοποιήστε τον αλγόριθμο <B>Node2vec</B> <a href="https://cs.stanford.edu/~jure/pubs/node2vec-kdd16.pdf">[3]</a>. Ο αλγόριθμος αντιστοιχίζει τους κόμβους του γράφου μέσω τυχαίων περιπάτων (biased 2nd order random walks) σε ένα χώρο μικρού αριθμού διαστάσεων (low dimensional space) διατηρώντας την μεταξύ τους γειτνίαση.
        <BR>Για την επιλογή των παραμέτρων: 
            <li>p - παράμετρος τυχαίων περιπάτων p που καθορίζει την πιθανότητα “1/p” επιστροφής στον κόμβο προέλευσης (source node), </li>
                <li>q - παράμετρος τυχαίων περιπάτων q που καθορίζει την πιθανότητα  “1/q” μετακίνησης σε κόμβο μακριά από τον κόμβο προέλευσης (source node), </li>
                 <li>dimensions - πλήθος διαστάσεων των Node2Vec embeddings, </li>
                     <li>num_walks - αριθμός περιπάτων από κάθε κόμβο, </li>
                         <li>walk_length - μήκος τυχαίου περιπάτου, </li>
                             <li>window_size - μέγεθος παραθύρου context για τον αλγόριθμο Word2Vec, </li><li>num_iter - αριθμός SGD επαναλήψεων (epochs), </li>
            <li>workers - αριθμός workers για τον Word2Vec,</li>
            χρησιμοποιήστε τις τιμές:   </html>
   
                   

 |   p   |   q   | dimensions|  num_walks  |walk_length |window_size|        workers            |
 |-------|-------|-----------|-------------|------------|-----------|---------------------------|
 |  1.0  |  1.0  |  128      |     10      |    80      |     10    |multiprocessing.cpu_count()|

In [32]:
# install node2vec with conda / pip
from node2vec import Node2Vec
import multiprocessing

#generate walks
node2vec_walker = Node2Vec(G_reduced, 
                dimensions=128, 
                walk_length=80, 
                num_walks=10, 
                p=1.0,
                q=1.0,
                workers=multiprocessing.cpu_count())

#train node2vec model
window_size=10
n2v_model = node2vec_walker.fit(window=window_size, min_count=1)

c:\Users\User\Downloads\DSML MsC\1st semester\Ανάλυση Κοινωνικών Δικτύων - Social Networks\Εργαστηριακές Ασκήσεις\2022\Social-Networks-Analysis\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Computing transition probabilities: 100%|██████████| 430/430 [00:00<00:00, 5375.12it/s]


<html><font>
    <li><B>4(ii).</B> Εφαρμόστε τον αλγόριθμο Random Forest Classifier, για να προβλέψετε τις συνδέσεις.</li><BR>
    <li><B>4(iii).</B> Υπολογίστε την ακρίβεια ($accuracy$) του μοντέλου. </li><BR></font></html>

In [49]:
df3 = copy.deepcopy(df)
df3.head()

,Edge,Link
0,"(343, 344)",1
1,"(32, 21)",1
2,"(338, 341)",1
3,"(19, 20)",1
4,"(68, 93)",1


In [50]:
node1_list = [x for x,y in df['Edge']]
node2_list = [y for x,y in df['Edge']]
df3['node_1'] = node1_list
df3['node_2'] = node2_list

df3.head()


,Edge,Link,node_1,node_2
0,"(343, 344)",1,343,344
1,"(32, 21)",1,32,21
2,"(338, 341)",1,338,341
3,"(19, 20)",1,19,20
4,"(68, 93)",1,68,93


In [51]:
#apply trained Node2Vec model on each node pair of the dataframe "unconnected+removables"
x = [(n2v_model.wv[str(i)]+n2v_model.wv[str(j)]) for i,j in zip(df3['node_1'], df3['node_2'])]

X is now a list of vectors (lists) with 128 elements/features each.

In [59]:
type(x), len(x), type(x[0]), len(x[0])

(list, 91900, numpy.ndarray, 128)

In [67]:
y = df['Link'].to_numpy()

In [77]:
X_train, X_test, y_train, y_test = train_test_split(np.array(x), y, test_size=0.33, random_state = 0)

In [79]:
print(f'Training')
print(f'\ttrain: {X_train.shape}, type: {X_train.dtype}')
print(f'\ttest : {X_test.shape}, type: {X_test.dtype}')

print(f'Testing')
print(f'\ttrain: {y_train.shape}, type: {y_train.dtype}')
print(f'\ttest : {y_test.shape}, type: {y_test.dtype}')

Training
	train: (61573, 128), type: float32
	test : (30327, 128), type: float32
Testing
	train: (61573,), type: int64
	test : (30327,), type: int64


In [80]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

classifier = RandomForestClassifier(max_depth=2, random_state=0) 
classifier.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [81]:
preds = classifier.predict(X_test)

In [85]:
acc = accuracy_score(y_test, preds)
pr = precision_score(y_test, preds)
rc = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)

print('Prediction metrics:')
print(f'Accuracy : {round(acc, 4)}\nPrecision: {round(pr, 4)}\nRecall   : {round(rc, 4)}\nF1-score : {round(f1, 4)}')

Prediction metrics:
Accuracy : 0.9985
Precision: 1.0
Recall   : 0.3478
F1-score : 0.5161


Here we can see again that the accuracy metric appears to be close to 1 given the imbalance in the two classes. Also, precision, is 1 which means that we have high confidence in our classifier's correctness when it predicts that a sample point belongs to the positive class. However, the recall metric is fairly low, which means that the True Positives over the Positives is low, i.e. out of all the positives, the classifier is able to discover only 34.5% of them and classify them correctly in the positive class. Then, the F1-score, which is the harmonic mean of precision and recall, is found to be around 52%, and provides a better metric for the evaluation of the predictive power of our classifier compared to accuracy in this imbalanced classes scenario. 

We can conclude here, that the results yieldied by the similarity-based classifier were slighly better (f1-score = 58%) than the embedding random walk approach (f1-score=52%), therefore we would use the first one for the task at hand in a production setup.

<html><font size=4> <B><U>Ερώτηση bonus (+10/100)</U><B>: </font>Για τους binary operators για learning των edge features με τον αλγόριθμο Node2Vec που αναφέρονται στο <a href="https://cs.stanford.edu/~jure/pubs/node2vec-kdd16.pdf">[3]</a> ((α) Average, (β) Hadamard, (γ) Weighted-L1 και (δ) Weighted-L2) τροποποιήστε τoν κώδικά σας και σημειώστε τις παρατηρήσεις σας ως προς την απόδοση του αλγορίθμου.</font></html>

In [ ]:
from sklearn.metrics import classification_report


def evaluate_n2v(op):
    y = df['link']
    X = np.stack(list(map(lambda x: op(n2v_model.wv[x[0]], n2v_model.wv[x[1]]), df['edges'])))
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    clf = RandomForestClassifier(max_depth=2, random_state=42).fit(X_train, y_train)
    preds = clf.predict(X_test)
    print(op.__name__)
    print(classification_report(y_test, preds))


def average(x, y):
    return (x + y) / 2


def hadamard(x, y):
    return x * y


def weighted_l1(x, y):
    return np.abs(x - y)


def weighted_l2(x, y):
    return (x - y)**2


# Kostas

def evaluate_rf_n2v(op):
    X = [op(word_2_vec.wv[u], word_2_vec.wv[v]) for u, v in df.index]
    y = df['is_edge']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=RANDOM_STATE)

    rf = RandomForestClassifier(max_depth=20, n_jobs=-1, random_state=RANDOM_STATE)
    rf.fit(X_train, y_train)
    preds = rf.predict(X_test)

    return classification_report(y_test, preds)


def average(x, y):
    return (x + y) / 2


def hadamard(x, y):
    return x * y


def l1(x, y):
    return np.abs(x - y)


def l2(x, y):
    return (x - y)**2


ops = average, hadamard, l1, l2
print(*(f'{op.__name__}\n{evaluate_rf_n2v(op)}' for op in ops), sep='\n')


In [91]:
n2v_model.wv

In [104]:
# binary operators

def default_op(u, v):
    return u + v

def average_op(u, v):
    return (u + v) / 2

def hadamard_op(u, v):
    return u * v

def l1_weighted_op(u, v):
    return np.abs(u - v)


def l2_weighted_op(u, v):
    return (u - v)**2

In [107]:
from sklearn.metrics import classification_report

def n2v_model_operator(bin_operator):

    X = [bin_operator(n2v_model.wv[(i)], n2v_model.wv[(j)]) for i,j in zip(df3['node_1'], df3['node_2'])]
    y = df['Link'].to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, test_size=0.33, random_state = 0)

    classifier = RandomForestClassifier(max_depth=2, random_state=0) 
    classifier.fit(X_train, y_train)
    
    preds = classifier.predict(X_test)
    print(classification_report(y_test, preds))  # show classification results

In [108]:
print('Default operator:')
n2v_model_operator(default_op)
print('__________________________________________________\n')
print('Average operator:')
n2v_model_operator(average_op)
print('__________________________________________________\n')
print('Hadamard operator:')
n2v_model_operator(hadamard_op)
print('__________________________________________________\n')
print('L1 weighted operator:')
n2v_model_operator(l1_weighted_op)
print('__________________________________________________\n')
print('L2 weighted operator:')
n2v_model_operator(l2_weighted_op)


Default operator:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     30258
           1       1.00      0.35      0.52        69

    accuracy                           1.00     30327
   macro avg       1.00      0.67      0.76     30327
weighted avg       1.00      1.00      1.00     30327

__________________________________________________

Average operator:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     30258
           1       1.00      0.35      0.52        69

    accuracy                           1.00     30327
   macro avg       1.00      0.67      0.76     30327
weighted avg       1.00      1.00      1.00     30327

__________________________________________________

Hadamard operator:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     30258
           1       1.00      0.35      0.52        69

    accuracy           

c:\Users\User\Downloads\DSML MsC\1st semester\Ανάλυση Κοινωνικών Δικτύων - Social Networks\Εργαστηριακές Ασκήσεις\2022\Social-Networks-Analysis\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\Downloads\DSML MsC\1st semester\Ανάλυση Κοινωνικών Δικτύων - Social Networks\Εργαστηριακές Ασκήσεις\2022\Social-Networks-Analysis\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\Downloads\DSML MsC\1st semester\Ανάλυση Κοινωνικών Δικτύων - Social Networks\Εργαστηριακές Ασκήσεις\2022\Soc

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     30258
           1       0.00      0.00      0.00        69

    accuracy                           1.00     30327
   macro avg       0.50      0.50      0.50     30327
weighted avg       1.00      1.00      1.00     30327

__________________________________________________

L2 weighted operator:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     30258
           1       0.00      0.00      0.00        69

    accuracy                           1.00     30327
   macro avg       0.50      0.50      0.50     30327
weighted avg       1.00      1.00      1.00     30327



c:\Users\User\Downloads\DSML MsC\1st semester\Ανάλυση Κοινωνικών Δικτύων - Social Networks\Εργαστηριακές Ασκήσεις\2022\Social-Networks-Analysis\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\Downloads\DSML MsC\1st semester\Ανάλυση Κοινωνικών Δικτύων - Social Networks\Εργαστηριακές Ασκήσεις\2022\Social-Networks-Analysis\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\Downloads\DSML MsC\1st semester\Ανάλυση Κοινωνικών Δικτύων - Social Networks\Εργαστηριακές Ασκήσεις\2022\Soc

<html><font size=4><B><U>Βιβλιογραφία:</U></B><BR></font>
<a href="https://dl.acm.org/doi/10.1145/2487788.2488173">[1]	Jérôme Kunegis. KONECT – The Koblenz Network Collection. In Proc. Int. Conf. on World Wide Web Companion, pages 1343–1350, 2013.</a><BR>
<a href="https://link.springer.com/chapter/10.1007/978-3-540-76298-0_52">[2]	Sören Auer, Christian Bizer, Georgi Kobilarov, Jens Lehmann, Richard Cyganiak, and Zachary Ives. DBpedia: A nucleus for a web of open data. In Proc. Int. Semant. Web Conf., pages 722–735, 2008.<BR>
<a href="https://cs.stanford.edu/~jure/pubs/node2vec-kdd16.pdf">[3] A. Grover, J. Leskovec, Node2vec: Scalable Feature Learning for Networks. ACM KDD, 2016.</a><BR>
 <a href="https://proceedings.neurips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf">[4] T.Mikolov, I.Sutskever, K.Chen, G.S.Corrado and J.Dean, Distributed representations of words and phrases and their compositionality. In NIPS, 2013.</a><BR></font></html>